# Natural Language Processing

## Exercise Sheet 8

In [1]:
import nltk

### Exercise 1

Write a <mark>recursive function</mark> to traverse a tree and return the <mark>depth of the tree</mark>, such that a tree with a single node would have <mark>depth zero</mark>. (Hint: the depth of a subtree is the maximum depth of its children, plus one.)
Test your function with the two trees produced by the `ChartParser` for the `groucho_grammar` and the sentence "I shot an elephant in my pajamas". The result can be verified with the `Tree.height()` function.

In [2]:
groucho_grammar = nltk.CFG.fromstring("""
S -> NP VP
PP -> P NP
NP -> Det N | Det N PP | 'I'
VP -> V NP | VP PP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot'
P -> 'in'
""")

sent = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']
parser = nltk.ChartParser(groucho_grammar)

for tree in parser.parse(sent):
    print(tree)

(S
  (NP I)
  (VP
    (VP (V shot) (NP (Det an) (N elephant)))
    (PP (P in) (NP (Det my) (N pajamas)))))
(S
  (NP I)
  (VP
    (V shot)
    (NP (Det an) (N elephant) (PP (P in) (NP (Det my) (N pajamas))))))


><font color='orange'>**Comments:** Below, I've compared the results to `Tree.height() - 1`, since the task was to give a tree having a single node <mark>depth zero</mark> and not one (as in the `Tree.height()` function)! Otherwise, we need to `return 1`.</font>

In [3]:
def tree_depth(tree):
    """ Recursive function to traverse a tree and return its depth. """
    if type(tree) == nltk.tree.Tree:
        return max(tree_depth(child) for child in tree) + 1
    return 0

In [4]:
# compute tree depths recursively using tree_depth function
for i, tree in enumerate(parser.parse(sent), start=1):
    depth = tree_depth(tree)
    print(f'Depth of tree {i}:', depth, f'({depth == tree.height()-1})')

Depth of tree 1: 5 (True)
Depth of tree 2: 6 (True)


In [5]:
help(nltk.tree.Tree)

Help on class Tree in module nltk.tree:

class Tree(builtins.list)
 |  Tree(node, children=None)
 |  
 |  A Tree represents a hierarchical grouping of leaves and subtrees.
 |  For example, each constituent in a syntax tree is represented by a single Tree.
 |  
 |  A tree's children are encoded as a list of leaves and subtrees,
 |  where a leaf is a basic (non-tree) value; and a subtree is a
 |  nested Tree.
 |  
 |      >>> from nltk.tree import Tree
 |      >>> print(Tree(1, [2, Tree(3, [4]), 5]))
 |      (1 2 (3 4) 5)
 |      >>> vp = Tree('VP', [Tree('V', ['saw']),
 |      ...                  Tree('NP', ['him'])])
 |      >>> s = Tree('S', [Tree('NP', ['I']), vp])
 |      >>> print(s)
 |      (S (NP I) (VP (V saw) (NP him)))
 |      >>> print(s[1])
 |      (VP (V saw) (NP him))
 |      >>> print(s[1,1])
 |      (NP him)
 |      >>> t = Tree.fromstring("(S (NP I) (VP (V saw) (NP him)))")
 |      >>> s == t
 |      True
 |      >>> t[1][1].set_label('X')
 |      >>> t[1][1].label()
 

### Exercise 2

Write a recursive function `bracketing(tree)` that produces a nested bracketing for a `tree`, leaving out the leaf nodes, and displaying the non-terminal labels after their subtrees. Consecutive categories should be separated by space. Test your function with the tree: 

In [4]:
from nltk.corpus import treebank
t = treebank.parsed_sents('wsj_0001.mrg')[0]
print(t)

(S
  (NP-SBJ
    (NP (NNP Pierre) (NNP Vinken))
    (, ,)
    (ADJP (NP (CD 61) (NNS years)) (JJ old))
    (, ,))
  (VP
    (MD will)
    (VP
      (VB join)
      (NP (DT the) (NN board))
      (PP-CLR (IN as) (NP (DT a) (JJ nonexecutive) (NN director)))
      (NP-TMP (NNP Nov.) (CD 29))))
  (. .))


In [5]:
def bracketing(tree):
    """ Recursive function that produces a nested bracketing for a tree. """
    
    # leaving out leave notes
    if tree_depth(tree) == 0: 
        return ""
    
    # consecutive categories are separated by space
    nested_string = ' '.join(bracketing(subtree) for subtree in tree)

    # produces a nested bracketing for a tree
    if tree_depth(tree) >= 2:
        nested_string = "[" + nested_string + "]"
        
    # displaying the non-terminal labels after their subtrees
    nested_string += tree.label()

    return nested_string

In [6]:
print(bracketing(t))

[[[NNP NNP]NP , [[CD NNS]NP JJ]ADJP ,]NP-SBJ [MD [VB [DT NN]NP [IN [DT JJ NN]NP]PP-CLR [NNP CD]NP-TMP]VP]VP .]S


It should produce the following output:

    [[[NNP NNP]NP , [[CD NNS]NP JJ]ADJP ,]NP-SBJ [MD [VB [DT NN]NP 
    [IN [DT JJ NN]NP]PP-CLR [NNP CD]NP-TMP]VP]VP .]S

### Exercise 3

Modify the functions `init_wfst()` and `complete_wfst()` so that the contents of each cell in the WFST is a set of non-terminal symbols rather than a single non-terminal. Test your function with the `groucho_grammar` and the sentence "I shot an elephant in my pajamas". 

><font color='orange'>**Notes:** I've commented out the old lines of code and added a `# NEW` to the new ones, in order to make it easier to compare the changes.</font>

In [7]:
groucho_grammar = nltk.CFG.fromstring("""
S -> NP VP
PP -> P NP
NP -> Det N | Det N PP | 'I'
VP -> V NP | VP PP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot'
P -> 'in'
""")

sent = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']

def init_wfst(tokens, grammar):
    numtokens = len(tokens)
    # wfst = [[None for i in range(numtokens+1)] for j in range(numtokens+1)]
    wfst = [[set() for i in range(numtokens+1)] for j in range(numtokens+1)]  # NEW
    for i in range(numtokens):
        productions = grammar.productions(rhs=tokens[i])
        # wfst[i][i+1] = productions[0].lhs()
        wfst[i][i+1].add(productions[0].lhs())  # NEW
    return wfst


def complete_wfst(wfst, tokens, grammar, trace=False):
    index = dict((p.rhs(), p.lhs()) for p in grammar.productions())
    numtokens = len(tokens)
    for span in range(2, numtokens+1):
        for start in range(numtokens+1-span):
            end = start + span
            for mid in range(start+1, end):
                # nt1, nt2 = wfst[start][mid], wfst[mid][end]
                for nt1 in list(wfst[start][mid]):  # NEW
                    for nt2 in list(wfst[mid][end]):  # NEW
                        if nt1 and nt2 and (nt1,nt2) in index:
                            # wfst[start][end] = index[(nt1,nt2)]
                            wfst[start][end].add(index[(nt1,nt2)])  # NEW
                            if trace:
                                print("[%s] %3s [%s] %3s [%s] ==> [%s] %3s [%s]" % \
                                (start, nt1, mid, nt2, end, start, index[(nt1,nt2)], end))
    return wfst


def display(wfst):
    print('\nWFST ' + ' '.join(("%-5d" % i) for i in range(1, len(wfst))))
    for i in range(len(wfst)-1):
        print("%d   " % i, end=" ")
        for j in range(1, len(wfst)):
            print("%-5s" % (wfst[i][j] or '.'), end=" ")
        print()

        
wfst0 = init_wfst(sent, groucho_grammar)
display(wfst0)
wfst = complete_wfst(wfst0, sent, groucho_grammar, True)
display(wfst)


WFST 1     2     3     4     5     6     7    
0    {NP}  .     .     .     .     .     .     
1    .     {V}   .     .     .     .     .     
2    .     .     {Det} .     .     .     .     
3    .     .     .     {N}   .     .     .     
4    .     .     .     .     {P}   .     .     
5    .     .     .     .     .     {Det} .     
6    .     .     .     .     .     .     {N}   
[2] Det [3]   N [4] ==> [2]  NP [4]
[5] Det [6]   N [7] ==> [5]  NP [7]
[1]   V [2]  NP [4] ==> [1]  VP [4]
[4]   P [5]  NP [7] ==> [4]  PP [7]
[0]  NP [1]  VP [4] ==> [0]   S [4]
[1]  VP [4]  PP [7] ==> [1]  VP [7]
[0]  NP [1]  VP [7] ==> [0]   S [7]

WFST 1     2     3     4     5     6     7    
0    {NP}  .     .     {S}   .     .     {S}   
1    .     {V}   .     {VP}  .     .     {VP}  
2    .     .     {Det} {NP}  .     .     .     
3    .     .     .     {N}   .     .     .     
4    .     .     .     .     {P}   .     {PP}  
5    .     .     .     .     .     {Det} {NP}  
6    .     .     .     .    

Change the line:

    NP -> Det N | Det N PP | 'I' 

in `groucho_grammar` to:

    NP -> Det N | NP PP | 'I' 

to verify in the trace of `complete_wfst()` that there are now two lines for `cell(1,7)`:

    [1]   V [2]  NP [7] ==> [1]  VP [7]
    [1]  VP [4]  PP [7] ==> [1]  VP [7]

In [8]:
groucho_grammar = nltk.CFG.fromstring("""
S -> NP VP
PP -> P NP
NP -> Det N | NP PP | 'I'
VP -> V NP | VP PP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot'
P -> 'in'
""")

wfst0 = init_wfst(sent, groucho_grammar)
display(wfst0)
wfst = complete_wfst(wfst0, sent, groucho_grammar, True)
display(wfst)


WFST 1     2     3     4     5     6     7    
0    {NP}  .     .     .     .     .     .     
1    .     {V}   .     .     .     .     .     
2    .     .     {Det} .     .     .     .     
3    .     .     .     {N}   .     .     .     
4    .     .     .     .     {P}   .     .     
5    .     .     .     .     .     {Det} .     
6    .     .     .     .     .     .     {N}   
[2] Det [3]   N [4] ==> [2]  NP [4]
[5] Det [6]   N [7] ==> [5]  NP [7]
[1]   V [2]  NP [4] ==> [1]  VP [4]
[4]   P [5]  NP [7] ==> [4]  PP [7]
[0]  NP [1]  VP [4] ==> [0]   S [4]
[2]  NP [4]  PP [7] ==> [2]  NP [7]
[1]   V [2]  NP [7] ==> [1]  VP [7]
[1]  VP [4]  PP [7] ==> [1]  VP [7]
[0]  NP [1]  VP [7] ==> [0]   S [7]

WFST 1     2     3     4     5     6     7    
0    {NP}  .     .     {S}   .     .     {S}   
1    .     {V}   .     {VP}  .     .     {VP}  
2    .     .     {Det} {NP}  .     .     {NP}  
3    .     .     .     {N}   .     .     .     
4    .     .     .     .     {P}   .     {PP}  
5   

Change the line:

    VP -> V NP | VP PP

in `groucho_grammar` to: 

    VP -> V NP
    VPC -> VP PP 

and check that `cell(1,7)` now contains `{VPC, VP}`.

In [9]:
groucho_grammar = nltk.CFG.fromstring("""
S -> NP VP
PP -> P NP
NP -> Det N | NP PP | 'I'
VP -> V NP
VPC -> VP PP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot'
P -> 'in'
""")

wfst0 = init_wfst(sent, groucho_grammar)
display(wfst0)
wfst = complete_wfst(wfst0, sent, groucho_grammar, True)
display(wfst)


WFST 1     2     3     4     5     6     7    
0    {NP}  .     .     .     .     .     .     
1    .     {V}   .     .     .     .     .     
2    .     .     {Det} .     .     .     .     
3    .     .     .     {N}   .     .     .     
4    .     .     .     .     {P}   .     .     
5    .     .     .     .     .     {Det} .     
6    .     .     .     .     .     .     {N}   
[2] Det [3]   N [4] ==> [2]  NP [4]
[5] Det [6]   N [7] ==> [5]  NP [7]
[1]   V [2]  NP [4] ==> [1]  VP [4]
[4]   P [5]  NP [7] ==> [4]  PP [7]
[0]  NP [1]  VP [4] ==> [0]   S [4]
[2]  NP [4]  PP [7] ==> [2]  NP [7]
[1]   V [2]  NP [7] ==> [1]  VP [7]
[1]  VP [4]  PP [7] ==> [1] VPC [7]
[0]  NP [1]  VP [7] ==> [0]   S [7]

WFST 1     2     3     4     5     6     7    
0    {NP}  .     .     {S}   .     .     {S}   
1    .     {V}   .     {VP}  .     .     {VP, VPC} 
2    .     .     {Det} {NP}  .     .     {NP}  
3    .     .     .     {N}   .     .     .     
4    .     .     .     .     {P}   .     {PP}  


Finally, change the line:

    S -> NP VP

in `groucho_grammar` to:

    S -> NP VP | NP VPC

and check that now there are two lines in the trace of `complete_wfst()` for the `cell(0,7)`:

    [0]  NP [1] VPC [7] ==> [0]   S [7]
    [0]  NP [1]  VP [7] ==> [0]   S [7]

In [10]:
groucho_grammar = nltk.CFG.fromstring("""
S -> NP VP | NP VPC
PP -> P NP
NP -> Det N | NP PP | 'I'
VP -> V NP
VPC -> VP PP
Det -> 'an' | 'my'
N -> 'elephant' | 'pajamas'
V -> 'shot'
P -> 'in'
""")

wfst0 = init_wfst(sent, groucho_grammar)
display(wfst0)
wfst = complete_wfst(wfst0, sent, groucho_grammar, True)
display(wfst)


WFST 1     2     3     4     5     6     7    
0    {NP}  .     .     .     .     .     .     
1    .     {V}   .     .     .     .     .     
2    .     .     {Det} .     .     .     .     
3    .     .     .     {N}   .     .     .     
4    .     .     .     .     {P}   .     .     
5    .     .     .     .     .     {Det} .     
6    .     .     .     .     .     .     {N}   
[2] Det [3]   N [4] ==> [2]  NP [4]
[5] Det [6]   N [7] ==> [5]  NP [7]
[1]   V [2]  NP [4] ==> [1]  VP [4]
[4]   P [5]  NP [7] ==> [4]  PP [7]
[0]  NP [1]  VP [4] ==> [0]   S [4]
[2]  NP [4]  PP [7] ==> [2]  NP [7]
[1]   V [2]  NP [7] ==> [1]  VP [7]
[1]  VP [4]  PP [7] ==> [1] VPC [7]
[0]  NP [1]  VP [7] ==> [0]   S [7]
[0]  NP [1] VPC [7] ==> [0]   S [7]

WFST 1     2     3     4     5     6     7    
0    {NP}  .     .     {S}   .     .     {S}   
1    .     {V}   .     {VP}  .     .     {VP, VPC} 
2    .     .     {Det} {NP}  .     .     {NP}  
3    .     .     .     {N}   .     .     .     
4    .     .

### Exercise 4

Modify the function `complete_wfst()` from Exercise 3 so that when a non-terminal symbol is added to a cell in the WFST, the content of the variable `mid` is also added, i.e. we add a tuple `(symbol, mid)`. In `init_wfst()`, use `(symbol, i+1)` instead. Change also the function `display()` accordingly. Test your implementation with the final grammar from Exercise 3 and the sentence "I shot an elephant in my pajamas". 

><font color='orange'>**Notes:** I've commented out the old lines of code (from Ex. 3) and added a `# NEW` to the new ones, in order to make it easier to compare the changes.</font>

In [11]:
def init_wfst(tokens, grammar):
    numtokens = len(tokens)
    wfst = [[set() for i in range(numtokens+1)] for j in range(numtokens+1)]  # NEW
    for i in range(numtokens):
        productions = grammar.productions(rhs=tokens[i])
        # wfst[i][i+1].add(productions[0].lhs())  # EX3
        wfst[i][i+1].add((productions[0].lhs(), i+1))  # NEW
    return wfst


def complete_wfst(wfst, tokens, grammar, trace=False):
    index = dict((p.rhs(), p.lhs()) for p in grammar.productions())
    numtokens = len(tokens)
    for span in range(2, numtokens+1):
        for start in range(numtokens+1-span):
            end = start + span
            for mid in range(start+1, end):
                for nt1, _ in list(wfst[start][mid]):  # NEW
                    for nt2, _ in list(wfst[mid][end]):  # NEW
                        if nt1 and nt2 and (nt1,nt2) in index:
                            wfst[start][end].add((index[(nt1,nt2)], mid))  # NEW
                            if trace:
                                print("[%s] %3s [%s] %3s [%s] ==> [%s] %3s [%s]" % \
                                (start, nt1, mid, nt2, end, start, index[(nt1,nt2)], end))
    return wfst


def display(wfst):
    print('\nWFST ' + ' '.join(("%-10d" % i) for i in range(1, len(wfst))))
    for i in range(len(wfst)-1):
        print("%d   " % i, end=" ")
        for j in range(1, len(wfst)):
            print("%-10s" % (wfst[i][j] or '.'), end=" ")
        print()

wfst0 = init_wfst(sent, groucho_grammar)
display(wfst0)
wfst = complete_wfst(wfst0, sent, groucho_grammar, True)
display(wfst)


WFST 1          2          3          4          5          6          7         
0    {(NP, 1)}  .          .          .          .          .          .          
1    .          {(V, 2)}   .          .          .          .          .          
2    .          .          {(Det, 3)} .          .          .          .          
3    .          .          .          {(N, 4)}   .          .          .          
4    .          .          .          .          {(P, 5)}   .          .          
5    .          .          .          .          .          {(Det, 6)} .          
6    .          .          .          .          .          .          {(N, 7)}   
[2] Det [3]   N [4] ==> [2]  NP [4]
[5] Det [6]   N [7] ==> [5]  NP [7]
[1]   V [2]  NP [4] ==> [1]  VP [4]
[4]   P [5]  NP [7] ==> [4]  PP [7]
[0]  NP [1]  VP [4] ==> [0]   S [4]
[2]  NP [4]  PP [7] ==> [2]  NP [7]
[1]   V [2]  NP [7] ==> [1]  VP [7]
[1]  VP [4]  PP [7] ==> [1] VPC [7]
[0]  NP [1]  VP [7] ==> [0]   S [7]
[0]  NP [1] 

It should produce the following output:

    WFST      1          2          3          4          5          6          7         
    0         {(NP, 1)}  .          .          {(S, 1)}   .          .          {(S, 1)}   
    1         .          {(V, 2)}   .          {(VP, 2)}  .          .          {(VP, 2), (VPC, 4)} 
    2         .          .          {(Det, 3)} {(NP, 3)}  .          .          {(NP, 4)}  
    3         .          .          .          {(N, 4)}   .          .          .          
    4         .          .          .          .          {(P, 5)}   .          {(PP, 5)}  
    5         .          .          .          .          .          {(Det, 6)} {(NP, 6)}  
    6         .          .          .          .          .          .          {(N, 7)}    

### Exercise 5

Use the extended WFST from Exercise 4 to retrace the parse trees for our example sentence "I shot an elephant in my pajamas''. Write a recursive function `retrace(WFST, tokens)` (the second parameter `tokens` contains the token list \['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas'\] for our example sentence). Start with `cell(0,7)` (or `cell(0,len(tokens))` in general) and use the information in `mid` to follow the productions to `cell(0,mid)` and `cell(mid,7)`, and so on. If we reach a terminal symbol, i.e. a `cell(i,i+1)`, the corresponding token from `tokens` shall be displayed. 


><font color='orange'>**Notes:** I've manually printed the statements to match the given format. Symbols which have already been printed are passed as `''`.</font>

In [13]:
def retrace(wfst, tokens, symbols=[], i=0, j=7):
    """ Recursive generator function to retrace the parse trees. """
    for symbol, mid in sorted(wfst[i][j], reverse=True):
        if i == j-1:
            # terminal symbol
            symbols = symbols + [symbol] + [tokens[i]]
            
            # format and print sequence
            for i, s in enumerate(symbols):
                if s == '':
                    print(s, end="\t")
                elif i == 0:
                    print("   %s" % s, end="\t")
                elif symbols[i-1] == '':
                    print("   %s" % s, end="\t")
                else:
                    print("-> %s" % s, end="\t")
            print()
            
        else:
            # follow productions of cell(i, mid) and cell(mid, j)
            retrace(wfst, tokens, symbols + [symbol], i, mid)
            retrace(wfst, tokens, [''] * len(symbols) + [''], mid, j)
    return None


retrace(wfst=wfst, tokens=sent)

   S	-> NP	-> I	
	   VPC	-> VP	-> V	-> shot	
			   NP	-> Det	-> an	
				   N	-> elephant	
		   PP	-> P	-> in	
			   NP	-> Det	-> my	
				   N	-> pajamas	
	   VP	-> V	-> shot	
		   NP	-> NP	-> Det	-> an	
				   N	-> elephant	
			   PP	-> P	-> in	
				   NP	-> Det	-> my	
					   N	-> pajamas	


The function should produce the following output:

    S    -> NP   -> I 
            VPC  -> VP   -> V    -> shot 
                            NP   -> Det  -> an 
                                    N    -> elephant 
                    PP   -> P    -> in 
                            NP   -> Det  -> my 
                                    N    -> pajamas 
            VP   -> V    -> shot 
                    NP   -> NP   -> Det  -> an 
                                    N    -> elephant 
                            PP   -> P    -> in 
                                    NP   -> Det  -> my 
                                            N    -> pajamas  

### Exercise 6

Process each tree of the Penn Treebank Corpus sample `nltk.corpus.treebank` and extract the productions with the help of `Tree.productions()`. Discard the productions that occur only once and those that are lexical (i.e. the right-hand side contains at least one terminal token). Productions with the same left-hand side can be collapsed using a dictionary with the left-hand sides as keys and sets of right-hand sides as values.

Print the value for the left-hand side 'NP' using the format: 

    DT JJS NN NN | DT VBG NN NN | DT NNP CD NN | DT NN NNS ...

><font color='orange'>**Notes:** I've used all trees in the `treebank.parse_sents()` list. At first I tried to filter by `fileid` but the number of trees was much lower. Moreover, I've used `FreqDist` to compute the number of occurances per production.</font>

In [14]:
# process each tree of the Penn Treebank Corpus sample and extract the productions
productions = [p for tree in treebank.parsed_sents() for p in tree.productions()]

# discard the productions that occur only once
productions = [p for p, count in nltk.FreqDist(productions).items() if count > 1]

# discard the productions that are lexical
productions = [p for p in productions if not p.is_lexical()]

# left-hand side can be collapsed using a dictionary
results = dict()
for p in productions:
    if p.lhs() in results.keys():
        results[p.lhs()].add(p.rhs())
    else:
        results[p.lhs()] = set(p.rhs())

# print the value for the left-hand side 'NP'
print(' | '.join(str(r).replace(',', '').replace('(', '').replace(')', '') 
                 for r in results.get(nltk.grammar.Nonterminal('NP'))))

DT JJ NN NN | PRP$ ADJP NN NN | DT JJ JJ NNS | NP CD NN | PRP$ NNP NN NN | JJ CD | NP NNS S | NP PP ADJP  VP . | DT UCP NN | NP PP-DIR PP-DIR | RB JJR | NP  SBAR-TMP | NP  NP | ADJP NN NNS | DT NAC-LOC NN NN | NP NN CC NN NN NN | DT NN CC NNS | DT NNP NNP NNS | NP '' PP | DT NNP NNPS | NP  NP  | PRP$ JJ NN NNS | NN NN | NNP NNPS NNP | ADJP JJ NN | DT NNP NNP NNP NN NN | NNP NN NN | DT NAC NNP | DT ADJP NN NN | NNP JJ NNS | DT VBG | NNP . | NP  PP  SBAR | NNP CC NNP POS | NP PP  PP | DT JJ NN CC NN NN | DT NN S | NP NP-LOC | CD NN NNS | DT NNP NNP CC NNP NNP | NP ADJP | CD | NP PP PP-LOC | NNPS NNP | DT JJ NNP NN | RB CD NNS | NNP NX | DT JJ DT | RB DT | DT JJS NN | NN  NN CC NN | PRP$ NNP NNP NN | CD NNS | ADJP NNS | ADJP NNP NNP NNP | ADJP NNP | JJ VBN NNS | ADJP NNP NNS | NP : NP | NNP NNP  NNP NNP CC NNP | DT JJ NX | NNP CC NNP | NP PP-LOC SBAR | PRP$ JJ ADJP NN | NNP NNP JJ NN | NP VP . | NP QP . | DT NN NNS | NP PP PP PP | NP ADVP | VB NNS | NP PP SBAR-TMP | NP  NP  NP  NP | NNS  